# Final Project for CS 6.338
# Emily Crabb

# Parallel molecular dyanmics simulation implementation in Julia

# SharedArray Version - Chunks

# Modified version for benchmarking (no saving outputs or plotting)

This version of the SharedArray parallel MD implementation is designed for benchmarking the actual MD implementation.  As such, the parameters have been set so that the results are not written to a file.  The code has also been altered so that the particles' positions are no longer saved periodically, so the positions are also not plotted.  These changes were to ensure the timing captures the most essential functions of the program, namely updating the particles' positions, velocities, accelerations, and forces.  The program performs the timing with various numbers of workers.  However, Julia should still be started with mulitple threads to truly test the impact of implementing parallelism.

This version of the MD code uses SharedArrays to implement parallelism.  The positions, velocities, accelerations, and forces are all stored in SharedArrays that each thread can access.  Then, in both the step_update() and find_force() functions, the matrices are conceptually broken up and the work is distributed among the threads.  The program will also add workers depending on how many threads Julia is started with.

This notebook implements a parallel version of a molecular dynamics simulation in the Julia programming language.  Notable features include

1) The option to read parameters (like number of simulation steps) and/or initial data (starting configuration: position, velocity, acceleration, forces) from external files.  These files must be in the same folder as this notebook and have the correct names (as specified in the code).

2) The option to directly specify the parameters in the notebook.  Note: These parameters are all constants, so one must restart the kernel to redefine them.

3) The option to save the parameters and output data to external files.

4) The option to model finite and infinite systems.

5) The ability to make finite systems periodic or non-periodic.

6) If the initial data / configuration is not specified in a file, it can be generated in the code.  For example, the starting positions are random within the specified box size.  The user can modify any of the initial conditions by altering the initialize() function.  

7) The user can also modify the form and strength of the forces by altering the find_forces() and gen_interaction() functions.

8) For 2 or 3 dimensional simulations, the system can be visually displayed in a plot.  For non-Windows systems, the user can also use the Interact package to manipulate the plot to see the movement of the particles in the system over time.  One of the parameters sets the frequency with which the program saves the particles' positions.

Note: The parameters are all constants, so one must restart the kernel to redefine them.

Note: If the parameters and/or initial data is read from files, the program assumes the files are compatible (i.e. the dimension in the dim.txt file matches the dimensions of the particles' positions, etc.).  This is not a problem if the user is restarting from previously saved files but could be a problem if the user has made changes to the files.

References:

For Julia help: The documentation at https://julialang.org/

For calculating forces: Article titled "Verlet integration" at https://www.saylor.org/site/wp-content/uploads/2011/06/MA221-6.1.pdf

For general layout of basic MD code: John Burkardt's website at https://people.sc.fsu.edu/~jburkardt/py_src/md/md.html

In [1]:
const params_from_file = true; # Whether input parameters should be read from file
const data_from_file = true; # Whether starting data should be read from file
const save_data = false; # Whether to save data to file so can restart

In [2]:
# Generate strengths of interactions between particle types
# Very simple but user could replace with anything they wanted

function gen_interaction(num_part_types)
    interaction_params = zeros(Float64, num_part_types,num_part_types)
    for i=1:num_part_types
        for j = 1:num_part_types
            if (i==j) # Self-interaction is randomly repulsive
                interaction_params[i,j] = -1*rand(Float64)
            elseif (i<j) # Others randomly attractive
                val = rand(Float64)
                interaction_params[i,j] = val
                interaction_params[j,i] = val
            end
        end        
    end
    
    return interaction_params
end

gen_interaction (generic function with 1 method)

In [3]:
# Global constants 
# Read from file if specified

if (params_from_file) # Read parameters in from file
    const number_of_steps = Int(readdlm("number_of_steps.txt")[1]); # Number of steps to execute in simulation
    const dim = Int(readdlm("dim.txt")[1]); # Dimensions of simulation
    const box_size = readdlm("box_size.txt")[1]; # Size of one side of box
    const finite_box = readdlm("finite_box.txt")[1]; # Whether box if finite or just where particles are initially placed
    const periodic = readdlm("periodic.txt")[1]; # Whether simulation is periodic
    const part_num = Int(readdlm("part_num.txt")[1]); # Number of particles in simulation
    const dt = readdlm("dt.txt")[1]; # Time step
    const num_part_types = Int(readdlm("num_part_types.txt")[1]); # Number of types of particles
    const interaction_params = readdlm("interaction_params.txt"); # Interations parameters for types of particles
    const mass_parts = readdlm("mass_parts.txt"); # Masses of types of particles
    const save_interval = Int(readdlm("save_interval.txt")[1]); # How often save position
else # Assign parameter values here
    const number_of_steps = 10000; # Number of steps to execute in simulation
    const dim = 3; # Dimensions of simulation
    const box_size = 100.0; # Size of one side of box
    const finite_box = true; # Whether box if finite or just where particles are initially placed
    const periodic = true; # Whether simulation is periodic
    const part_num = 100; # Number of particles in simulation
    const dt = 0.01; # Time step
    const num_part_types = 2; # Number of types of particles
    const interaction_params = gen_interaction(num_part_types); # Interations parameters for types of particles
    const mass_parts = rand(Float64, num_part_types); # Masses of types of particles
    const save_interval = 10; # How often save position
end

10

In [4]:
# Initialize position, velocity, acceleration, and particle type
# Can read in from file or generate in code
# If generate in code, currently start with zero velocity and acceleration
# If generate in code, currently start with random positions and randomly assigned particle types

function initialize(part_num, dim, box_size, num_part_types, data_from_file)

    if (data_from_file) # Read data in from file
        part_types = readdlm("part_types.txt", Int)
        pos = readdlm("saved_positions.txt")
        pos = convert(SharedArray,pos) # Store in SharedArray so can use @parallel
        vel = readdlm("saved_velocities.txt")
        vel = convert(SharedArray,vel) # Store in SharedArray so can use @parallel
        acc = readdlm("saved_accelerations.txt")
        acc = convert(SharedArray,acc) # Store in SharedArray so can use @parallel
    else # Generate data
        pos = box_size*rand(Float64, part_num, dim) # Initialized to be randomly placed within a box
        pos = convert(SharedArray,pos) # Store in SharedArray so can use @parallel

        vel = zeros(Float64, part_num, dim) # Initialized to zero
        vel = convert(SharedArray,vel) # Store in SharedArray so can use @parallel
        acc = zeros(Float64, part_num, dim) # Initialized to zero
        acc = convert(SharedArray,acc) # Store in SharedArray so can use @parallel

        part_types = rand(1:num_part_types, part_num) # Randomly assign type of each particle
    end
        
    return pos, vel, acc, part_types
end

initialize (generic function with 1 method)

In [5]:
# Taken from Julia documentation

@everywhere function myrange(q::SharedArray)
    idx = indexpids(q)
    if idx == 0 # This worker is not assigned a piece
       return 1:0, 1:0
    end
    nchunks = length(procs(q))
    splits = [round(Int, s) for s in linspace(0,size(q,2),nchunks+1)]
    1:size(q,1), splits[idx]+1:splits[idx+1]
end

In [6]:
@everywhere function step_update_chunk!(part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt, box_size, finite_box, periodic)

    for i in myrange(pos)[1] # For every particle
        mass = mass_parts[part_types[i]]
        for j in myrange(pos)[2] # For each dimension
            pos[i,j] = pos[i,j] + vel[i,j]*dt + 0.5*acc[i,j]*dt^2 # x(t+Δt) = x(t) + v(t)Δt + 1/2*a(t)(Δt)^2
            vel[i,j] = vel[i,j] + 0.5*(acc[i,j] + force[i,j]/mass)*dt # v(t+Δt) = v(t) + 1/2*(a(t)+a(t+Δt))Δt
            acc[i,j] = force[i,j]/mass # a = F/m
            
            if (finite_box) # If finite box, check are still inside and correct if not
                if (periodic) # For periodic, just change position to be in box
                    if (pos[i,j] < 0) # If no longer in box
                        pos[i,j] = box_size + (pos[i,j] % box_size)
                    elseif (pos[i,j] > box_size) # If no longer in box
                        pos[i,j] = pos[i,j] % box_size                        
                    end
                else # If not periodic, more complicated - reflects off walls
                    if (pos[i,j] < 0) # If no longer in box
                        pos[i,j] = -1*(pos[i,j])
                    elseif (pos[i,j] > box_size) # If no longer in box
                        pos[i,j] = box_size - pos[i,j]
                    end
                    vel[i,j] = -1*(vel[i,j])
                    acc[i,j] = -1*(acc[i,j])
                end
            end
        end
    end
        
end

In [7]:
# Update position, velocity, and acceleration using Velocity Verlet Algorithm
# Can deal with infinite and finite systems
# For finite system, can be periodic or can reflect off walls

function step_update(part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt, box_size, finite_box, periodic)
    
    @sync begin
        for p in procs(pos)
            @async remotecall_wait(step_update_chunk!, p, part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt, box_size, finite_box, periodic)
        end
    end
    
    # return pos, vel, acc # No need to return b/c is passed by reference
end

step_update (generic function with 1 method)

In [8]:
@everywhere function find_force_chunk!(part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts, periodic, box_size, force)
    
    for i in myrange(pos)[1] # For every particle
        mass = mass_parts[part_types[i]]
        for k = 1:part_num # Contribution from every other particle
            if (i != k) # No self-interaction
                for j in myrange(pos)[2] # For each dimension
                    int_strength = interaction_params[part_types[i],part_types[k]] # Strength of interaction between particles
                    if (pos[i,j] > pos[k,j])
                       int_strength = -1*int_strength # Reverses direction of force if positions flopped
                    end

                    # Find distance between particles
                    dist = 0.0
                    if (periodic) # If periodic, check whether a periodic distance is shorter
                        dist = abs(pos[i,j] - pos[k,j])
                        if (pos[i,j] < pos[k,j])
                            new_dist = abs(box_size + pos[i,j] - pos[k,j])
                            if new_dist > dist
                                dist = new_dist
                                int_strength = -1*int_strength # Reverses direction of force
                            end
                        else
                            new_dist = abs(box_size + pos[k,j] - pos[i,j])
                            if new_dist > dist
                                dist = new_dist
                                int_strength = -1*int_strength # Reverses direction of force
                            end
                        end
                        else # Otherwise, just regular distance
                        dist = abs(pos[i,j] - pos[k,j])
                    end

                    force[i,j] += int_strength / dist^2 # 1/r^2 interaction
                end
            end
        end
    end
     
end

In [9]:
# Find force on each particle
# 1/r^2 interactions: Is very simple but user can replace with anything they want

function find_force(part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts, periodic, box_size)
    
    #force = zeros(part_num, dim)
    #force = convert(SharedArray,force)
    
    force = SharedArray{Float64,2}(part_num,dim) # Store forces in SharedArray so can use @parallel
    
    @sync begin
        for p in procs(pos)
            @async remotecall_wait(find_force_chunk!, p, part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts, periodic, box_size, force)
        end
    end

    return force
end

find_force (generic function with 1 method)

In [10]:
# Write each variable to its own output file in current diretory
# Decided to write each variable to own file because is very easy to read values in and restart

function write_output(number_of_steps, dim, box_size, finite_box, periodic, part_num, dt, num_part_types, interaction_params, mass_parts, save_interval, part_types, saved_positions, saved_velocities, saved_accelerations, saved_forces) 
    writedlm("number_of_steps.txt", number_of_steps)
    writedlm("dim.txt", dim)
    writedlm("box_size.txt", box_size)
    writedlm("finite_box.txt", finite_box)
    writedlm("periodic.txt", periodic)
    writedlm("part_num.txt", part_num)
    writedlm("dt.txt", dt)
    writedlm("num_part_types.txt", num_part_types)
    writedlm("interaction_params.txt", interaction_params)
    writedlm("mass_parts.txt", mass_parts)
    writedlm("save_interval.txt", save_interval)
    writedlm("part_types.txt", part_types)
    writedlm("saved_positions.txt", saved_positions[size(saved_positions,1),:,:])
    writedlm("saved_velocities.txt", saved_velocities[size(saved_velocities,1),:,:])
    writedlm("saved_accelerations.txt", saved_accelerations[size(saved_accelerations,1),:,:])
    writedlm("saved_forces.txt", saved_forces[size(saved_forces,1),:,:])
end

write_output (generic function with 1 method)

In [11]:
# Main body of program where positions, velocities, accelerations, and forces are updated
# This is the part of the program that is timed

function main(number_of_steps, part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt, box_size, finite_box, periodic, interaction_params)

    for i = 1:number_of_steps
        step_update(part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt, box_size, finite_box, periodic) # Update
        force = find_force(part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts, periodic, box_size) # Find new forces
    end
    
end

main (generic function with 1 method)

In [12]:
using BenchmarkTools

In [13]:
println("This program is using ", Threads.nthreads(), " threads")

This program is using 8 threads


In [14]:
# Use multiple processes / workers
# Optimal number will depend on machine

desired_num_procs = 1 # Try with just 1
curr_num_procs = nprocs()

if (curr_num_procs < desired_num_procs) # Use four processes
    addprocs(desired_num_procs - curr_num_procs)
end
println("This program is using ", nprocs(), " processes")
println("This program is using ", nworkers(), " workers")

# Initialization
pos, vel, acc, part_types = initialize(part_num, dim, box_size, num_part_types, data_from_file) # Initialize
if (data_from_file)
    force = readdlm("saved_forces.txt")
else
    force = find_force(part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts, periodic, box_size) # Find forces on particles
end

# Benchmarking
@benchmark main(number_of_steps, part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt, box_size, finite_box, periodic, interaction_params)

This program is using 1 processes
This program is using 1 workers


BenchmarkTools.Trial: 
  memory estimate:  89.44 GiB
  allocs estimate:  1000371688
  --------------
  minimum time:     414.588 s (1.57% GC)
  median time:      414.588 s (1.57% GC)
  mean time:        414.588 s (1.57% GC)
  maximum time:     414.588 s (1.57% GC)
  --------------
  samples:          1
  evals/sample:     1

In [15]:
# Use multiple processes / workers
# Optimal number will depend on machine

desired_num_procs = 4 # Try with 4
curr_num_procs = nprocs()

if (curr_num_procs < desired_num_procs) # Use four processes
    addprocs(desired_num_procs - curr_num_procs);
end

3-element Array{Int64,1}:
 2
 3
 4

In [16]:
# Taken from Julia documentation

@everywhere function myrange(q::SharedArray)
    idx = indexpids(q)
    if idx == 0 # This worker is not assigned a piece
       return 1:0, 1:0
    end
    nchunks = length(procs(q))
    splits = [round(Int, s) for s in linspace(0,size(q,2),nchunks+1)]
    1:size(q,1), splits[idx]+1:splits[idx+1]
end

In [17]:
@everywhere function step_update_chunk!(part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt, box_size, finite_box, periodic)

    for i in myrange(pos)[1] # For every particle
        mass = mass_parts[part_types[i]]
        for j in myrange(pos)[2] # For each dimension
            pos[i,j] = pos[i,j] + vel[i,j]*dt + 0.5*acc[i,j]*dt^2 # x(t+Δt) = x(t) + v(t)Δt + 1/2*a(t)(Δt)^2
            vel[i,j] = vel[i,j] + 0.5*(acc[i,j] + force[i,j]/mass)*dt # v(t+Δt) = v(t) + 1/2*(a(t)+a(t+Δt))Δt
            acc[i,j] = force[i,j]/mass # a = F/m
            
            if (finite_box) # If finite box, check are still inside and correct if not
                if (periodic) # For periodic, just change position to be in box
                    if (pos[i,j] < 0) # If no longer in box
                        pos[i,j] = box_size + (pos[i,j] % box_size)
                    elseif (pos[i,j] > box_size) # If no longer in box
                        pos[i,j] = pos[i,j] % box_size                        
                    end
                else # If not periodic, more complicated - reflects off walls
                    if (pos[i,j] < 0) # If no longer in box
                        pos[i,j] = -1*(pos[i,j])
                    elseif (pos[i,j] > box_size) # If no longer in box
                        pos[i,j] = box_size - pos[i,j]
                    end
                    vel[i,j] = -1*(vel[i,j])
                    acc[i,j] = -1*(acc[i,j])
                end
            end
        end
    end
        
end

In [18]:
@everywhere function find_force_chunk!(part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts, periodic, box_size, force)
    
    for i in myrange(pos)[1] # For every particle
        mass = mass_parts[part_types[i]]
        for k = 1:part_num # Contribution from every other particle
            if (i != k) # No self-interaction
                for j in myrange(pos)[2] # For each dimension
                    int_strength = interaction_params[part_types[i],part_types[k]] # Strength of interaction between particles
                    if (pos[i,j] > pos[k,j])
                       int_strength = -1*int_strength # Reverses direction of force if positions flopped
                    end

                    # Find distance between particles
                    dist = 0.0
                    if (periodic) # If periodic, check whether a periodic distance is shorter
                        dist = abs(pos[i,j] - pos[k,j])
                        if (pos[i,j] < pos[k,j])
                            new_dist = abs(box_size + pos[i,j] - pos[k,j])
                            if new_dist > dist
                                dist = new_dist
                                int_strength = -1*int_strength # Reverses direction of force
                            end
                        else
                            new_dist = abs(box_size + pos[k,j] - pos[i,j])
                            if new_dist > dist
                                dist = new_dist
                                int_strength = -1*int_strength # Reverses direction of force
                            end
                        end
                        else # Otherwise, just regular distance
                        dist = abs(pos[i,j] - pos[k,j])
                    end

                    force[i,j] += int_strength / dist^2 # 1/r^2 interaction
                end
            end
        end
    end
     
end

In [19]:
println("This program is using ", nprocs(), " processes")
println("This program is using ", nworkers(), " workers")

# Initialization
pos, vel, acc, part_types = initialize(part_num, dim, box_size, num_part_types, data_from_file) # Initialize
if (data_from_file)
    force = readdlm("saved_forces.txt")
else
    force = find_force(part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts, periodic, box_size) # Find forces on particles
end

# Benchmarking
@benchmark main(number_of_steps, part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt, box_size, finite_box, periodic, interaction_params)

This program is using 4 processes
This program is using 3 workers


BenchmarkTools.Trial: 
  memory estimate:  175.00 MiB
  allocs estimate:  2002945
  --------------
  minimum time:     555.314 s (0.00% GC)
  median time:      555.314 s (0.00% GC)
  mean time:        555.314 s (0.00% GC)
  maximum time:     555.314 s (0.00% GC)
  --------------
  samples:          1
  evals/sample:     1

In [20]:
# Use multiple processes / workers
# Optimal number will depend on machine

desired_num_procs = 10 # Try with 10
curr_num_procs = nprocs()

if (curr_num_procs < desired_num_procs) # Use 10 processes
    addprocs(desired_num_procs - curr_num_procs);
end

6-element Array{Int64,1}:
  5
  6
  7
  8
  9
 10

In [21]:
# Taken from Julia documentation

@everywhere function myrange(q::SharedArray)
    idx = indexpids(q)
    if idx == 0 # This worker is not assigned a piece
       return 1:0, 1:0
    end
    nchunks = length(procs(q))
    splits = [round(Int, s) for s in linspace(0,size(q,2),nchunks+1)]
    1:size(q,1), splits[idx]+1:splits[idx+1]
end

In [22]:
@everywhere function step_update_chunk!(part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt, box_size, finite_box, periodic)

    for i in myrange(pos)[1] # For every particle
        mass = mass_parts[part_types[i]]
        for j in myrange(pos)[2] # For each dimension
            pos[i,j] = pos[i,j] + vel[i,j]*dt + 0.5*acc[i,j]*dt^2 # x(t+Δt) = x(t) + v(t)Δt + 1/2*a(t)(Δt)^2
            vel[i,j] = vel[i,j] + 0.5*(acc[i,j] + force[i,j]/mass)*dt # v(t+Δt) = v(t) + 1/2*(a(t)+a(t+Δt))Δt
            acc[i,j] = force[i,j]/mass # a = F/m
            
            if (finite_box) # If finite box, check are still inside and correct if not
                if (periodic) # For periodic, just change position to be in box
                    if (pos[i,j] < 0) # If no longer in box
                        pos[i,j] = box_size + (pos[i,j] % box_size)
                    elseif (pos[i,j] > box_size) # If no longer in box
                        pos[i,j] = pos[i,j] % box_size                        
                    end
                else # If not periodic, more complicated - reflects off walls
                    if (pos[i,j] < 0) # If no longer in box
                        pos[i,j] = -1*(pos[i,j])
                    elseif (pos[i,j] > box_size) # If no longer in box
                        pos[i,j] = box_size - pos[i,j]
                    end
                    vel[i,j] = -1*(vel[i,j])
                    acc[i,j] = -1*(acc[i,j])
                end
            end
        end
    end
        
end

In [23]:
@everywhere function find_force_chunk!(part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts, periodic, box_size, force)
    
    for i in myrange(pos)[1] # For every particle
        mass = mass_parts[part_types[i]]
        for k = 1:part_num # Contribution from every other particle
            if (i != k) # No self-interaction
                for j in myrange(pos)[2] # For each dimension
                    int_strength = interaction_params[part_types[i],part_types[k]] # Strength of interaction between particles
                    if (pos[i,j] > pos[k,j])
                       int_strength = -1*int_strength # Reverses direction of force if positions flopped
                    end

                    # Find distance between particles
                    dist = 0.0
                    if (periodic) # If periodic, check whether a periodic distance is shorter
                        dist = abs(pos[i,j] - pos[k,j])
                        if (pos[i,j] < pos[k,j])
                            new_dist = abs(box_size + pos[i,j] - pos[k,j])
                            if new_dist > dist
                                dist = new_dist
                                int_strength = -1*int_strength # Reverses direction of force
                            end
                        else
                            new_dist = abs(box_size + pos[k,j] - pos[i,j])
                            if new_dist > dist
                                dist = new_dist
                                int_strength = -1*int_strength # Reverses direction of force
                            end
                        end
                        else # Otherwise, just regular distance
                        dist = abs(pos[i,j] - pos[k,j])
                    end

                    force[i,j] += int_strength / dist^2 # 1/r^2 interaction
                end
            end
        end
    end
     
end

In [ ]:
println("This program is using ", nprocs(), " processes")
println("This program is using ", nworkers(), " workers")

# Initialization
pos, vel, acc, part_types = initialize(part_num, dim, box_size, num_part_types, data_from_file) # Initialize
if (data_from_file)
    force = readdlm("saved_forces.txt")
else
    force = find_force(part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts, periodic, box_size) # Find forces on particles
end

# Benchmarking
@benchmark main(number_of_steps, part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt, box_size, finite_box, periodic, interaction_params)

This program is using 10 processes
This program is using 9 workers
